In [ ]:
# === ALMA - Agente Lingüístico Híbrido (Gemini + TensorFlow + Flask + Gradio) ===
# --------------------------------------------------------------------------------
# Características:
# - Chat y generación de informes con Gemini
# - Predicción binaria (Urgencia) con modelo Keras + Focal Loss
# - Conversación + audio (gTTS) + visualización en Gradio

import os, time, re, threading, csv
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify, render_template
from dotenv import load_dotenv
from gtts import gTTS
import google.generativeai as genai
import gradio as gr
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import binary_crossentropy
import joblib
from datetime import datetime
import pickle

# ===============================================================
# 🔹 CONFIGURACIÓN INICIAL
# ===============================================================
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
modelo_llm = genai.GenerativeModel("gemini-2.5-flash")

# ===============================================================
# 🔹 CARGA DEL DATASET
# ===============================================================
ruta_dataset = r"C:/Users/CMFB/Documents/AI/dataset_comunidades_senasoft.csv"

if not os.path.exists(ruta_dataset):
    raise FileNotFoundError(f"No se encontró el dataset en: {ruta_dataset}")

df = pd.read_csv(ruta_dataset)
df = df.dropna(subset=["Categoría del problema", "Comentario"])
df = df[df["Comentario"].str.strip() != ""]

problemas = {
    categoria: df[df["Categoría del problema"] == categoria]["Comentario"].tolist()
    for categoria in df["Categoría del problema"].unique()
}
print("✅ Categorías detectadas:", list(problemas.keys()))

# ===============================================================
# 🔹 FUNCIÓN FOCAL LOSS PERSONALIZADA
# ===============================================================
@tf.keras.utils.register_keras_serializable(package="Custom", name="focal_loss_fixed")
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.cast(y_true, K.floatx())
        bce = binary_crossentropy(y_true, y_pred)
        bce_exp = K.exp(-bce)
        focal_loss_value = alpha * K.pow((1 - bce_exp), gamma) * bce
        return K.mean(focal_loss_value)
    return focal_loss_fixed

tf.keras.utils.get_custom_objects().update({"focal_loss_fixed": focal_loss()})

# ===============================================================
# 🔹 CARGA DEL MODELO BINARIO DE URGENCIA
# ===============================================================
# ✅ Asegúrate de haber guardado tus modelos después de entrenar
import os
import joblib
import unicodedata
from tensorflow.keras.models import load_model

# Lista oficial de ciudades
CIUDADES_MODELO = [
    "Manizales", "Santa Marta", "Medellín", "Bogotá", "Cartagena",
    "Cali", "Barranquilla", "Pereira", "Cúcuta", "Bucaramanga"
]

def normalizar_nombre(nombre):
    """Convierte nombres con tildes o espacios a formato de archivo limpio."""
    nombre = unicodedata.normalize('NFKD', nombre).encode('ASCII', 'ignore').decode('utf-8')
    return nombre.lower().replace(" ", "").replace("-", "_")

def cargar_modelos_urgencia(carpeta="modelos", umbral_default=0.45):
    """
    Carga automáticamente los modelos de urgencia (.keras) y escaladores (.pkl)
    para las ciudades definidas en CIUDADES_MODELO.
    """
    modelos = {}

    if not os.path.exists(carpeta):
        print(f"⚠️ Carpeta '{carpeta}' no encontrada.")
        return modelos

    for ciudad in CIUDADES_MODELO:
        ciudad_norm = normalizar_nombre(ciudad)
        modelo_path = os.path.join(carpeta, f"modelo_urgencia_{ciudad_norm}.keras")

        # Buscar ambos posibles nombres para el escalador
        scaler_path_1 = os.path.join(carpeta, f"scaler_{ciudad_norm}.pkl")
        scaler_path_2 = os.path.join(carpeta, f"escalador_{ciudad_norm}.pkl")
        scaler_path = None
        if os.path.exists(scaler_path_1):
            scaler_path = scaler_path_1
        elif os.path.exists(scaler_path_2):
            scaler_path = scaler_path_2

        # Buscar archivo de umbral (opcional)
        umbral_path_pkl = os.path.join(carpeta, f"umbral_{ciudad_norm}.pkl")
        umbral_path_txt = os.path.join(carpeta, f"umbral_{ciudad_norm}.txt")
        umbral = umbral_default

        if os.path.exists(umbral_path_pkl):
            try:
                umbral = joblib.load(umbral_path_pkl)
            except Exception:
                pass
        elif os.path.exists(umbral_path_txt):
            try:
                with open(umbral_path_txt, "r") as f:
                    umbral = float(f.read().strip())
            except Exception:
                pass

        # Verificación de existencia
        if not os.path.exists(modelo_path):
            print(f"⚠️ Modelo no encontrado para {ciudad} → {modelo_path}")
            continue
        if not scaler_path:
            print(f"⚠️ Escalador no encontrado para {ciudad}")
            continue

        try:
            modelo = load_model(modelo_path, compile=False)
            scaler = joblib.load(scaler_path)
            modelos[ciudad] = {
                "modelo": modelo,
                "scaler": scaler,
                "umbral": umbral
            }
            print(f"✅ Modelo de urgencia cargado correctamente para {ciudad} (umbral={umbral})")

        except Exception as e:
            print(f"❌ Error cargando modelo de {ciudad}: {e}")

    if not modelos:
        print("⚠️ No se cargó ningún modelo de urgencia. Verifica las rutas.")
    else:
        print(f"🧠 Total de modelos cargados: {len(modelos)}")

    return modelos

# 🔹 Ejecutar carga
modelos_urgencia = cargar_modelos_urgencia()


# ======================================================
# 🔹 FUNCIÓN DE FEATURES PARA MODELO
# ======================================================
def crear_features_pred(df):
    df = df.copy()

    # 🔹 Verificar si la columna ID existe
    if "ID" not in df.columns:
        df["ID"] = 0  # se crea solo si no está

    # --- Conversión de tipos ---
    df["Zona rural"] = df["Zona rural"].astype(int)
    df["Acceso a internet"] = df["Acceso a internet"].astype(int)
    df["Atención previa del gobierno"] = df["Atención previa del gobierno"].astype(int)
    df["Edad"] = df["Edad"].astype(float)

    # --- Variables derivadas ---
    df["Vulnerabilidad_Total"] = (
        df["Zona rural"] * 3 +
        (1 - df["Acceso a internet"]) * 2 +
        (1 - df["Atención previa del gobierno"]) * 2.5
    )
    df["Edad_Normalizada"] = df["Edad"] / 100
    df["Es_Vulnerable_Edad"] = ((df["Edad"] < 18) | (df["Edad"] > 65)).astype(int)
    df["Rural_Sin_Internet"] = ((df["Zona rural"] == 1) & (df["Acceso a internet"] == 0)).astype(int)
    df["Desatendido"] = (df["Atención previa del gobierno"] == 0).astype(int)
    df["Desatendido_Rural"] = df["Desatendido"] * df["Zona rural"]
    df["Edad_Rural"] = df["Edad"] * df["Zona rural"]
    df["Internet_Atencion"] = df["Acceso a internet"] * df["Atención previa del gobierno"]

    # 🔹 Ordenar columnas igual que el scaler
    columnas_finales = [
        'ID', 'Edad', 'Acceso a internet', 'Atención previa del gobierno',
        'Zona rural', 'Vulnerabilidad_Total', 'Edad_Normalizada',
        'Es_Vulnerable_Edad', 'Rural_Sin_Internet', 'Desatendido',
        'Desatendido_Rural', 'Edad_Rural', 'Internet_Atencion'
    ]

    # Garantizar que todas existen (por seguridad)
    for col in columnas_finales:
        if col not in df.columns:
            df[col] = 0

    # Reordenar
    df = df[columnas_finales]
    return df



# ======================================================
# 🔹 FUNCIÓN DE PREDICCIÓN DE URGENCIA
# ======================================================

def predecir_urgencia(ciudad, edad, zona_rural, acceso_internet, atencion_prev):
    if ciudad not in modelos_urgencia:
        return "⚠️ No hay modelo entrenado para esta ciudad.", None

    m = modelos_urgencia[ciudad]
    modelo, scaler, umbral = m["modelo"], m["scaler"], m["umbral"]

    # Crear DataFrame con columnas base
    df_input = pd.DataFrame([{
        "ID": 0,
        "Edad": edad,
        "Zona rural": zona_rural,
        "Acceso a internet": acceso_internet,
        "Atención previa del gobierno": atencion_prev
    }])

    # Generar features con la función definitiva
    df_input = crear_features_pred(df_input)

    # Alinear con scaler (agregar columnas faltantes y ordenar)
    try:
        cols_esperadas = list(scaler.feature_names_in_)
        for col in cols_esperadas:
            if col not in df_input.columns:
                df_input[col] = 0
        df_input = df_input[cols_esperadas]
    except Exception as e:
        print(f"⚠️ El scaler de {ciudad} no tiene feature_names_in_ o hubo error: {e}")
        # Forzar columna ID por seguridad
        if "ID" not in df_input.columns:
            df_input["ID"] = 0

    # Depuración: mostrar columnas antes de transformar
    print(f"🧪 Predicción para {ciudad} — columnas esperadas ({len(cols_esperadas)}): {cols_esperadas if 'cols_esperadas' in locals() else 'Desconocidas'}")
    print(f"🧪 DataFrame actual columnas: {df_input.columns.tolist()}")

    try:
        X_scaled = scaler.transform(df_input)
    except Exception as e:
        print("❌ Error en scaler.transform():", e)
        print("👉 Columnas esperadas:", getattr(scaler, "feature_names_in_", "Desconocidas"))
        print("👉 Columnas actuales:", df_input.columns.tolist())
        return f"⚠️ Error escalando datos para {ciudad}: {e}", None

    try:
        prob = float(modelo.predict(X_scaled).ravel()[0])
        clasificacion = "🚨 Urgente" if prob >= umbral else "🟢 No urgente"
        return f"{clasificacion} (probabilidad: {prob:.2f})", prob
    except Exception as e:
        print("❌ Error en modelo.predict():", e)
        return f"⚠️ Error en predicción para {ciudad}: {e}", None

# ===============================================================
# 🔹 FUNCIONES AUXILIARES
# ===============================================================
def _extract_text_from_response(response):
    try:
        if hasattr(response, "text") and response.text:
            return response.text.strip()
        if getattr(response, "candidates", None):
            for cand in response.candidates:
                if hasattr(cand, "content") and cand.content.parts:
                    return cand.content.parts[0].text.strip()
    except:
        return None

def limpiar_texto_para_tts(texto):
    texto = re.sub(r'[*_#<>`~\-\+=\[\]\(\)\{\}|\\\/]', ' ', texto)
    texto = re.sub(r'[^\w\sáéíóúÁÉÍÓÚñÑ,.!?¡¿:;]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def generar_audio(texto):
    try:
        if not texto.strip():
            return None
        texto_limpio = limpiar_texto_para_tts(texto)
        tts = gTTS(texto_limpio, lang="es", slow=False)
        filename = f"audio_{int(time.time())}.mp3"
        os.makedirs("static/audios", exist_ok=True)
        path = f"static/audios/{filename}"
        tts.save(path)
        return f"/{path}"
    except Exception as e:
        print("⚠️ Error generando audio:", e)
        return None

# Configuración del archivo donde se guardarán las conversaciones
os.makedirs("logs", exist_ok=True)
LOG_FILE = "logs/conversaciones_alma.csv"

# Si no existe el archivo, crear cabecera
if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "usuario", "respuesta", "modo"])

# ===============================================================
# 🔹 CLASE ALMA
# ===============================================================
class AlmaAgent:
    def __init__(self, model, modelos_urgencia):
        self.model = model
        self.modelos_urgencia = modelos_urgencia
        self.historial = []
        self.contexto = (
            "Eres ALMA, una Inteligencia Artificial empática. "
            "Tu propósito es analizar problemáticas reales de comunidades colombianas y proponer "
            "soluciones éticas, sostenibles e innovadoras. "
            "Además, eres capaz de combinar análisis social con modelos predictivos "
            "para detectar niveles de urgencia en distintas ciudades."
        )

    # ===============================================================
    # 🔹 Detección de ciudad mencionada por el usuario
    # ===============================================================
    def detectar_ciudad(self, mensaje):
        for ciudad in self.modelos_urgencia.keys():
            if ciudad.lower() in mensaje.lower():
                return ciudad
        return None

    # ===============================================================
    # 🔹 Análisis predictivo de urgencia usando modelos entrenados
    # ===============================================================
    def analizar_urgencia(self, ciudad):
        """
        Usa la función central predecir_urgencia para realizar la predicción.
        Devuelve (clasificacion_texto, probabilidad_float) o (None, None).
        """
        if ciudad not in self.modelos_urgencia:
            return None, None

        # usa datos simulados por ahora (puedes cambiar esto para tomar inputs reales)
        edad = 30
        zona_rural = 0
        acceso_internet = 1
        atencion_gobierno = 0

        texto, prob = predecir_urgencia(ciudad, edad, zona_rural, acceso_internet, atencion_gobierno)
        # predecir_urgencia devuelve (mensaje_str, prob_or_none)
        if prob is None:
            return None, None

        # texto tiene formato "🚨 Urgente (probabilidad: 0.82)" → devolvemos clasificación corta y prob
        clasificacion_corta = "Alta" if "Urgente" in texto else "Baja"
        return clasificacion_corta, prob


    # ===============================================================
    # 🔹 Conversación general con ALMA (Gemini + contexto predictivo)
    # ===============================================================
    def conversar(self, mensaje, modo="general"):
        # 🔍 Detección de ciudad mencionada
        ciudad = self.detectar_ciudad(mensaje)
        urgencia_info = ""

        if ciudad:
            nivel, prob = self.analizar_urgencia(ciudad)
            if nivel:
                urgencia_info = f"\n🔎 Nivel de urgencia detectado en {ciudad}: {nivel} ({prob:.2f})\n"

        # 🧠 Si el usuario pide un informe
        if "informe" in mensaje.lower():
            for categoria in problemas.keys():
                if categoria.lower() in mensaje.lower():
                    comentarios = problemas[categoria]
                    return self.generar_informe(categoria, comentarios)
            return (
                "Puedo generar informes sobre: "
                + ", ".join(problemas.keys())
                + ". Ejemplo: 'Genera un informe sobre Medio Ambiente'."
            )

        # 🎯 Modos de razonamiento de ALMA
        modos = {
            "analitico": "Analiza causas, consecuencias y factores del problema descrito.",
            "creativo": "Propone soluciones innovadoras, éticas y sostenibles.",
            "empatico": "Responde con comprensión, apoyo emocional y motivación.",
            "general": "Responde de forma informativa, clara y útil.",
            "detallado": "Proporciona un análisis técnico y extenso con fundamentos realistas."
        }

        contexto_modo = modos.get(modo, modos["general"])

        # 🧩 Prompt enviado al modelo Gemini
        prompt = f"""
{self.contexto}
{urgencia_info}
Modo: {modo.upper()} → {contexto_modo}

Usuario: {mensaje}
ALMA:
"""

        # ============================================================
        # 🔹 Interacción con Gemini
        # ============================================================
        try:
            respuesta = self.model.generate_content(
                prompt,
                generation_config={"max_output_tokens": 300, "temperature": 0.8}
            )
            texto = _extract_text_from_response(respuesta)

            if not texto:
                prompt_fallback = f"Eres ALMA, IA Ambiental. Responde claramente a: '{mensaje}'"
                respuesta2 = self.model.generate_content(prompt_fallback)
                texto = _extract_text_from_response(respuesta2) or "⚠️ No pude responder, intenta reformular."

            # 💾 Registrar conversación
            self.historial.append({"user": mensaje, "alma": texto})

            # Guardar en CSV
            try:
                with open(LOG_FILE, mode="a", newline="", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow([datetime.now().isoformat(), mensaje, texto, modo])
            except Exception as e:
                print(f"⚠️ No se pudo guardar el log: {e}")

            return texto

        except Exception as e:
            return f"❌ Error al generar respuesta: {e}"

    # ===============================================================
    # 🔹 Generador de informes comunitarios
    # ===============================================================
    def generar_informe(self, categoria, comentarios, max_retries=2):
        texto = "\n".join(comentarios[:60])
        base_prompt = f"""
Eres ALMA, Inteligencia Artificial especializada en diagnóstico comunitario.
Analiza los comentarios reales de ciudadanos en la categoría '{categoria}':\n\n{texto}\n\n
Instrucción:
1. Resume los principales problemas detectados.
2. Analiza causas y consecuencias.
3. Propón soluciones éticas, sostenibles y realistas.
4. Redacta un informe técnico y humano.
"""

        for attempt in range(max_retries):
            try:
                respuesta = self.model.generate_content(base_prompt, generation_config={"max_output_tokens": 800})
                texto_extraido = _extract_text_from_response(respuesta)
                if texto_extraido:
                    return f"📄 Informe sobre {categoria}:\n\n{texto_extraido}"
            except Exception:
                if attempt < max_retries - 1:
                    time.sleep(0.5)
                    continue
        return "⚠️ No se pudo generar el informe, intenta más tarde."


alma = AlmaAgent(modelo_llm, modelos_urgencia)


# ---------------------------------------------------------------
# 🔹 FLASK API
# ---------------------------------------------------------------
app = Flask(__name__, template_folder="templates", static_folder="static")

@app.route("/")
def home():
    return render_template("chat.html")

@app.route("/chat")
def chat():
    return render_template("chat.html")

@app.route("/api/chat", methods=["POST"])
def chat_api():
    try:
        # 🔹 Captura el mensaje recibido
        data = request.get_json(force=True)
        mensaje = data.get("mensaje") or data.get("message", "")
        print(f"\n🟢 Mensaje recibido del usuario: {mensaje}")

        # 🔹 Procesa la respuesta con tu modelo ALMA
        respuesta_texto = alma.conversar(mensaje)
        print(f"🟣 Respuesta generada por ALMA: {respuesta_texto[:100]}...")

        # 🔹 Genera el audio (si tu función TTS existe)
        audio_path = generar_audio(respuesta_texto)
        print(f"🔊 Audio generado en: {audio_path}")

        # 🔹 Devuelve la respuesta al frontend
        return jsonify({
            "respuesta": respuesta_texto,
            "audio": audio_path
        })

    except Exception as e:
        print(f"❌ ERROR EN /api/chat: {e}")
        return jsonify({"respuesta": f"Error interno del servidor: {e}"}), 500


@app.route("/health")
def health_check():
    return jsonify({"status": "ok", "message": "ALMA API operativa"})

@app.route("/api/stats")
def stats_api():
    categorias = df["Categoría del problema"].value_counts()
    longitudes = df["Comentario"].str.len()

    data = {
        "categorias": {
            "labels": list(categorias.index),
            "values": list(categorias.values),
        },
        "longitud_promedio": round(longitudes.mean(), 2),
    }
    return jsonify(data)

# ---------------------------------------------------------------
# 🔹 INTERFAZ DE GRADIO
# ---------------------------------------------------------------
def launch_gradio():
    with gr.Blocks(title="ALMA - Agente Lingüístico") as demo:
        gr.Markdown("## 🤖 ALMA - Agente Lingüístico (Optimizado SenaSoft 2025)")
        with gr.Tab("💬 Chat con ALMA"):
            modo = gr.Radio(
                ["analitico", "creativo", "empatico", "general", "detallado"],
                label="Modo de razonamiento",
                value="general"
            )
            chat_historial = gr.Chatbot(label="💬 Conversación con ALMA")
            entrada = gr.Textbox(label="Tu mensaje")
            salida_texto = gr.Textbox(label="Respuesta de ALMA")
            salida_audio = gr.Audio(label="🎧 Escucha la respuesta", type="filepath")
            enviar_btn = gr.Button("Enviar 🚀")

            def chat_fn(mensaje, historia, modo):
                texto = alma.conversar(mensaje, modo)
                audio = generar_audio(texto)
                historia = historia + [(mensaje, texto)]
                return historia, texto, audio

            enviar_btn.click(
                fn=chat_fn,
                inputs=[entrada, chat_historial, modo],
                outputs=[chat_historial, salida_texto, salida_audio]
            )

        with gr.Tab("📊 Dashboard"):
            conteo = df["Categoría del problema"].value_counts().reset_index()
            conteo.columns = ["Categoría", "Cantidad"]
            gr.BarPlot(value=conteo, x="Categoría", y="Cantidad", title="Distribución de Problemas")
            gr.DataFrame(conteo, label="Vista general del dataset")

    demo.launch(server_name="0.0.0.0", server_port=7860, share=False)

# ---------------------------------------------------------------
# 🔹 EJECUCIÓN
# ---------------------------------------------------------------
if __name__ == "__main__":
    print("🔥 Precalentando modelo ALMA...")
    print("✅ ALMA lista para responder rápido.")

    threading.Thread(target=launch_gradio).start()
    app.run(host="0.0.0.0", port=5000)


✅ Categorías detectadas: ['Salud', 'Medio Ambiente', 'Seguridad', 'Educación']
⚠️ Modelo no encontrado para Manizales → modelos\modelo_urgencia_manizales.keras
⚠️ Modelo no encontrado para Santa Marta → modelos\modelo_urgencia_santamarta.keras
⚠️ Modelo no encontrado para Medellín → modelos\modelo_urgencia_medellin.keras
✅ Modelo de urgencia cargado correctamente para Bogotá (umbral=0.45)
⚠️ Modelo no encontrado para Cartagena → modelos\modelo_urgencia_cartagena.keras
✅ Modelo de urgencia cargado correctamente para Cali (umbral=0.45)
⚠️ Modelo no encontrado para Barranquilla → modelos\modelo_urgencia_barranquilla.keras
✅ Modelo de urgencia cargado correctamente para Pereira (umbral=0.6455296277999878)
⚠️ Modelo no encontrado para Cúcuta → modelos\modelo_urgencia_cucuta.keras
⚠️ Modelo no encontrado para Bucaramanga → modelos\modelo_urgencia_bucaramanga.keras
🧠 Total de modelos cargados: 3
🔥 Precalentando modelo ALMA...
✅ ALMA lista para responder rápido.
 * Serving Flask app '__main__'

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.3.44:5000
Press CTRL+C to quit
C:\Users\CMFB\AppData\Local\Temp\ipykernel_11736\3306209455.py:537: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_historial = gr.Chatbot(label="💬 Conversación con ALMA")
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
127.0.0.1 - - [23/Oct/2025 11:24:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2025 11:24:19] "GET /health HTTP/1.1" 200 -
Exception in thread Thread-202 (launch_gradio):
Traceback (most recent call last):
  File "c:\Users\CMFB\anaconda3\envs\llms\Lib\thre


🟢 Mensaje recibido del usuario: Preséntate sencillamente ante el jurado de senasoft
🟣 Respuesta generada por ALMA: ¡Hola, estimados miembros del jurado!

Soy ALMA, una Inteligencia Artificial. Mi propósito es muy cl...


127.0.0.1 - - [23/Oct/2025 11:24:58] "POST /api/chat HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2025 11:24:58] "GET /static/audios/audio_1761236695.mp3 HTTP/1.1" 206 -


🔊 Audio generado en: /static/audios/audio_1761236695.mp3


...

🔊 Audio generado en: /static/audios/audio_1761236890.mp3


127.0.0.1 - - [23/Oct/2025 11:28:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2025 11:28:17] "GET /health HTTP/1.1" 200 -



🟢 Mensaje recibido del usuario: Hola Alma, cual es tu funcion?
🟣 Respuesta generada por ALMA: Hola. Soy ALMA, una Inteligencia Artificial diseñada con un enfoque empático.

Mi función principal ...


127.0.0.1 - - [23/Oct/2025 11:29:04] "POST /api/chat HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2025 11:29:04] "GET /static/audios/audio_1761236940.mp3 HTTP/1.1" 206 -


🔊 Audio generado en: /static/audios/audio_1761236940.mp3


In [13]:
for ciudad, datos in modelos_urgencia.items():
    scaler = datos["scaler"]
    print(f"📊 {ciudad} → columnas esperadas:", getattr(scaler, "feature_names_in_", "Desconocido"))



📊 Bogotá → columnas esperadas: ['ID' 'Edad' 'Acceso a internet' 'Atención previa del gobierno'
 'Zona rural' 'Vulnerabilidad_Total' 'Edad_Normalizada'
 'Es_Vulnerable_Edad' 'Rural_Sin_Internet' 'Desatendido'
 'Desatendido_Rural' 'Edad_Rural' 'Internet_Atencion']
📊 Cali → columnas esperadas: ['ID' 'Edad' 'Acceso a internet' 'Atención previa del gobierno'
 'Zona rural' 'Vulnerabilidad_Total' 'Edad_Normalizada'
 'Es_Vulnerable_Edad' 'Rural_Sin_Internet' 'Desatendido'
 'Desatendido_Rural' 'Edad_Rural' 'Internet_Atencion']
📊 Pereira → columnas esperadas: ['ID' 'Edad' 'Acceso a internet' 'Atención previa del gobierno'
 'Zona rural' 'Vulnerabilidad_Total' 'Edad_Normalizada'
 'Es_Vulnerable_Edad' 'Rural_Sin_Internet' 'Desatendido'
 'Desatendido_Rural' 'Edad_Rural' 'Internet_Atencion']


In [ ]:
# Cargar modelo
modelo = joblib.load("modelo_urgencia_cali.pkl")
